# single-userコンテナ起動確認（JupyterHub REST API利用）

## 概要

[Jupyterhub REST API](https://jupyterhub.readthedocs.io/en/latest/howto/rest.html)を利用し、ユーザのログイン～single-user notebook serverコンテナ起動の確認を行います。  

* 前提
    * Jupyterhub管理者のトークンが利用可能であること
* 実行方法
    * 各セルの説明に従って、実行してください
* 活用方法
    * 指定したユーザ分のコンテナが同時起動可能であることを確認できます。
    * 「801-資源消費量可視化.ipynb」ノートブックを利用して、動作確認中の資源消費量を確認できます。

### UnitGroup名

JupyterHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'JupyterHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

In [ ]:
target_nodes = f'{ugroup_name}_worker'

!ansible {target_nodes} -m ping

UnitGroup名に対応する group_varsファイルが存在していることを確認します。

In [ ]:
!test -f group_vars/{ugroup_name}

## パラメータの設定

JupyterHubシステムに関する各種設定項目を指定します。

ここで設定された項目はansible変数として保存され、ファイルのmanagerノードへの配置時に各ファイルに反映されます。

### 必須設定項目  

各種パラメータを指定します。  
初期状態で指定されているものは、基本的に変更不要です。

実行用の資材等を配置するディレクトリを指定します。

In [ ]:
work_dir = '/srv/jupyterhub/spawntest'

テスト用ユーザのグループIDを指定します。

In [ ]:
group_id_num = 601

jupyterhub REST APIを利用するためのトークンを指定します。  
トークンは、Jupyterhubに管理者としてログインし、tokenページ（`/hub/token`）で発行します。

In [ ]:
jupyterhub_token = 

### テスト用ユーザの指定

テスト用ユーザの情報を指定します。  
テスト後にホームディレクトリ等を削除するため、実際に利用されることのないユーザ情報を指定してください。

In [ ]:
# 個別に指定する場合
test_users = [
    {'username': 'testuser1', 'uid_num': 801},
    {'username': 'testuser2', 'uid_num': 802},
    {'username': 'testuser3', 'uid_num': 803},
]
# 人数を指定する場合
# user_count = 10
# test_users = [f'testuser{i}' for i in range(user_count)]

### 実行

#### 事前確認

テスト対象のサーバにアクセスできることを確認します。

In [ ]:
# selenium実行環境からmoodleへのアクセス
!ansible {target_hub} -m shell \
    -a "curl http://localhost:8000/hub/health -o /dev/null -w '%{{http_code}}\n' -s"

#### セットアップ

必要な資材の配置を行います。

まず、ファイルを格納するディレクトリを作成

In [ ]:
!ansible {target_hub} -b -m file -a \
    'path={work_dir} state=directory owner={{{{ansible_user}}}}'

ファイルの配置をチェックモードで試行

In [ ]:
!ansible {target_hub} -CDv -m synchronize \
    -a 'src=../tools/jupyterhub_api dest={work_dir}'

実際にファイルを配置

In [ ]:
!ansible {target_hub} -Dv -m synchronize \
    -a 'src=../tools/jupyterhub_api dest={work_dir}'

#### 実行  

テスト用ユーザのセットアップを行い、各ユーザ用のsingle-user serverコンテナの起動を行います。

テスト用ユーザのセットアップを行います。  
具体的には、以下の設定を行います。

- LDAPサーバへのユーザ登録
- Jupyterhubへのユーザ登録
- ホームディレクトリの作成

In [ ]:
for user in test_users:
    # ユーザセットアップ(ldap, homedir) 
    !ansible {target_hub} -m shell \
        -a "sudo bash {work_dir}/jupyterhub_api/setup_jupyterhub_testuser.sh {user['username']} {user['uid_num']} {group_id_num} {ugroup_name} {jupyterhub_token}"

single-user serverコンテナの起動（spawn）を行います。

In [ ]:
for user in test_users:
    !ansible {target_hub} -m shell \
        -a "sudo bash {work_dir}/jupyterhub_api/spawn.sh {user['username']} {user['uid_num']} {group_id_num} {jupyterhub_token} &"

#### 終了  

single-user serverコンテナのシャットダウンと、ユーザのホームディレクトリの削除を行います。

Docker swarmを利用しているため、サービスを停止します。

In [ ]:
for user in test_users:
    !ansible {target_hub} -m shell \
        -a "docker service rm jupyter-{user['username']}" || true

テストで使用したユーザ情報やディレクトリを削除します。

In [ ]:
for user in test_users:
    # ユーザセットアップ(ldap, homedir) 
    !ansible {target_hub} -m shell \
        -a "sudo bash {work_dir}/jupyterhub_api/cleanup_testuser.sh {user['username']} {user['uid_num']} {group_id_num} {ugroup_name} {jupyterhub_token}"